<a href="https://colab.research.google.com/github/MMLL168/ASR-Automatic-Speech-Recognition/blob/main/PromptLibrary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title ✈️ Prompt Library v6.0 (Ultimate UI)
# @markdown 🟢 支援：複選清單 (Multi-select)、自動串接、大量資料處理
# @markdown <br>
# @markdown #### 👇 點擊下方連結觀看教學並訂閱
# @markdown [![YouTube](https://img.shields.io/badge/YouTube-Watch%20Tutorial-red?style=for-the-badge&logo=youtube)](https://youtu.be/QCm8gAg9vqg?si=Dyoeo-FCNmZ5mWfd)

import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, Javascript
import requests
import json
import time

# --- ☁️ 雲端總控中心 ---
MASTER_INDEX_URL = "https://gist.githubusercontent.com/weihua-studio/051d3a6a6a50ef1690166b9502212c5e/raw/0cc983bef094bbf773da7ab78111efb0ff469264/library_index.json"

FALLBACK_LIBRARIES = {"⚠️ 離線預設包": "https://gist.githubusercontent.com/weihua-wang/raw/prompts.json"}

class CloudCopilot:
    def __init__(self):
        self.libraries = {}
        self.current_db = {}
        self.input_widgets = {}
        self.setup_ui()
        self.load_master_index()

    def setup_ui(self):
        # UI 樣式優化 (包含 YouTube 按鈕樣式)
        style = """
        <style>
            .app-wrapper { font-family: 'Segoe UI', sans-serif; max-width: 1000px; margin: 0 auto; background: #fff; border-radius: 12px; border: 1px solid #e5e7eb; box-shadow: 0 10px 15px -3px rgba(0, 0, 0, 0.1); overflow: hidden; }
            .header-bar { background: linear-gradient(135deg, #0f172a 0%, #334155 100%); color: white; padding: 16px 24px; display: flex; justify-content: space-between; align-items: center; }
            .section-label { font-size: 11px; font-weight: 800; color: #64748b; letter-spacing: 1px; margin-bottom: 8px; text-transform: uppercase; margin-top: 15px; }
            .multi-select-tip { font-size: 11px; color: #3b82f6; margin-top: -5px; margin-bottom: 5px; }

            /* YouTube 按鈕樣式 */
            .yt-btn {
                background-color: #FF0000;
                color: white !important;
                padding: 6px 14px;
                border-radius: 20px;
                text-decoration: none;
                font-size: 13px;
                font-weight: 600;
                display: flex;
                align-items: center;
                gap: 6px;
                transition: all 0.2s;
                border: 1px solid rgba(255,255,255,0.2);
            }
            .yt-btn:hover {
                background-color: #cc0000;
                transform: translateY(-1px);
                box-shadow: 0 4px 12px rgba(255,0,0,0.3);
            }
            .status-badge {
                font-size: 12px;
                background: rgba(255,255,255,0.1);
                padding: 4px 12px;
                border-radius: 20px;
                backdrop-filter: blur(4px);
            }
        </style>
        """

        # 🔥 設定你的 YouTube 影片連結
        youtube_link = "https://youtu.be/QCm8gAg9vqg?si=Dyoeo-FCNmZ5mWfd"

        # 建構 Header HTML
        header_content = f"""
        <div class='app-wrapper'>
            <div class='header-bar'>
                <div style='display:flex; flex-direction:column;'>
                    <div style='font-size:20px; font-weight:600; display:flex; align-items:center; gap:8px;'>
                        ✈️ Prompt Copilot
                        <span style='font-size:12px; background:#3b82f6; padding:2px 8px; border-radius:4px; opacity:0.9;'>v6.0 Ultimate</span>
                    </div>
                    <div style='font-size:11px; opacity:0.8; margin-top:4px;'>Developed by Weihua AI Studio</div>
                </div>
                <div style='display:flex; align-items:center; gap:12px;'>
                    <a href='{youtube_link}' target='_blank' class='yt-btn'>
                        <svg viewBox="0 0 24 24" width="16" height="16" fill="currentColor"><path d="M23.498 6.186a3.016 3.016 0 0 0-2.122-2.136C19.505 3.545 12 3.545 12 3.545s-7.505 0-9.377.505A3.017 3.017 0 0 0 .502 6.186C0 8.07 0 12 0 12s0 3.93.502 5.814a3.016 3.016 0 0 0 2.122 2.136c1.871.505 9.376.505 9.376.505s7.505 0 9.377-.505a3.015 3.015 0 0 0 2.122-2.136C24 15.93 24 12 24 12s0-3.93-.502-5.814zM9.545 15.568V8.432L15.818 12l-6.273 3.568z"/></svg>
                        觀看教學影片
                    </a>
                    <div id='status' class='status-badge'>Connecting...</div>
                </div>
            </div>
        </div>
        """

        self.header_html = widgets.HTML(style + header_content)

        self.lib_dd = widgets.Dropdown(description='📚 模組庫:', layout=widgets.Layout(width='99%'), style={'description_width': 'initial'})
        self.role_dd = widgets.Dropdown(description='📂 類別:', layout=widgets.Layout(width='99%'))
        self.task_dd = widgets.Dropdown(description='⚡ 任務:', layout=widgets.Layout(width='99%'))

        self.inputs_box = widgets.VBox(layout=widgets.Layout(padding='10px 0'))
        self.result_area = widgets.Textarea(layout=widgets.Layout(width='100%', height='450px'))

        self.copy_btn = widgets.Button(description='Copy Prompt', icon='copy', button_style='primary', layout=widgets.Layout(width='100%', margin='10px 0 0 0'))
        self.copy_btn.on_click(self.on_copy_click)

        self.lib_dd.observe(self.on_library_change, names='value')
        self.role_dd.observe(self.on_role_change, names='value')
        self.task_dd.observe(self.on_task_change, names='value')

    def load_master_index(self):
        try:
            r = requests.get(f"{MASTER_INDEX_URL}?t={int(time.time())}", timeout=5)
            if r.status_code == 200:
                self.libraries = r.json()
                status_txt = "🟢 Online"
            else: raise Exception
        except:
            self.libraries = FALLBACK_LIBRARIES
            status_txt = "🟡 Offline"

        self.lib_dd.options = list(self.libraries.keys())
        self.header_html.value = self.header_html.value.replace("Connecting...", status_txt)
        if self.libraries: self.on_library_change({'new': list(self.libraries.keys())[0]})

    def on_library_change(self, change):
        if not change['new']: return
        try:
            r = requests.get(f"{self.libraries[change['new']]}?t={int(time.time())}", timeout=5)
            if r.status_code == 200:
                self.current_db = r.json()
                self.refresh_roles_and_force_render()
        except Exception as e: self.result_area.value = f"Error: {e}"

    def refresh_roles_and_force_render(self):
        if 'roles' not in self.current_db: return
        roles = list(self.current_db['roles'].keys())
        self.role_dd.options = roles
        if roles:
            self.role_dd.value = roles[0]
            self.on_role_change({'new': roles[0]})

    def on_role_change(self, change):
        role = change['new']
        if role and role in self.current_db['roles']:
            tasks = list(self.current_db['roles'][role].keys())
            self.task_dd.options = tasks
            if tasks:
                self.task_dd.value = tasks[0]
                self.on_task_change({'new': tasks[0]})

    def on_task_change(self, change):
        role, task = self.role_dd.value, self.task_dd.value
        if role and task:
            self.render_inputs(self.current_db['roles'][role][task])

    def render_inputs(self, task_data):
        self.input_widgets = {}
        ui = []
        if "description" in task_data:
            ui.append(widgets.HTML(f"<div style='color:#64748b; font-size:13px; margin-bottom:12px;'>ℹ️ {task_data['description']}</div>"))

        for k, v in task_data['vars'].items():
            # 🔥 核心升級：偵測 __multi 後綴
            if k.endswith('__multi') and isinstance(v, list):
                clean_k = k.replace('__multi', '') # 顯示時去掉後綴
                ui.append(widgets.HTML(f"<div class='section-label'>{clean_k} (可複選: 按住 Ctrl/Cmd 點擊)</div>"))
                w = widgets.SelectMultiple(options=v, value=[v[0]], description='', rows=8)
                w.layout = widgets.Layout(width='99%')
            elif isinstance(v, list):
                w = widgets.Dropdown(options=v, value=v[0], description=k)
                w.layout = widgets.Layout(width='99%')
                w.style = {'description_width': 'initial'}
            else:
                w = widgets.Text(value=str(v), description=k)
                w.layout = widgets.Layout(width='99%')

            w.observe(self.update_result, names='value')
            self.input_widgets[k] = w
            if not k.endswith('__multi'): # Dropdown 已經有 label 了，SelectMultiple 上面手動加了 label
                ui.append(widgets.VBox([w], layout=widgets.Layout(margin='0 0 10px 0')))
            else:
                ui.append(widgets.VBox([w], layout=widgets.Layout(margin='0 0 10px 0')))

        self.inputs_box.children = ui
        self.update_result(None)

    def update_result(self, change):
        try:
            role, task = self.role_dd.value, self.task_dd.value
            tmpl = self.current_db['roles'][role][task]['template']
            vals = {}
            for k, w in self.input_widgets.items():
                if isinstance(w.value, tuple): # 處理複選結果
                    vals[k] = "、".join(w.value) # 自動用頓號連接
                else:
                    vals[k] = w.value
            self.result_area.value = tmpl.format(**vals)
        except: pass

    def on_copy_click(self, b):
        safe_content = json.dumps(self.result_area.value)
        display(Javascript(f"navigator.clipboard.writeText({safe_content})"))
        b.description = "Copied! ✅"; time.sleep(1); b.description = "Copy Prompt"

    def display(self):
        left = widgets.VBox([widgets.HTML("<div class='section-label'>CONFIG</div>"), self.lib_dd, self.role_dd, self.task_dd, widgets.HTML("<hr style='border-top:1px dashed #cbd5e1; margin:15px 0;'>"), widgets.HTML("<div class='section-label'>VARIABLES</div>"), self.inputs_box], layout=widgets.Layout(width='45%', padding='20px'))
        right = widgets.VBox([widgets.HTML("<div class='section-label'>RESULT</div>"), self.result_area, self.copy_btn], layout=widgets.Layout(width='55%', padding='20px', background_color='#f8fafc'))
        display(self.header_html, widgets.VBox([widgets.HBox([left, right])], layout=widgets.Layout(border='1px solid #ddd', border_top='none', border_radius='0 0 12px 12px')))

clear_output(); app = CloudCopilot(); app.display()